# Camera projection points
---

In [1]:
import numpy as np
import cv2
import os

def get_LeftImages(i):
    return cv2.imread('./images/img2/'+str(i)+'.png')

def get_RightImages(i):
    return cv2.imread('./images/img3/'+str(i)+'.png')

def get_ground_truth():
    file = 'poses.txt'
    return np.genfromtxt(file, delimiter=' ',dtype=None)

def get_transformation_matrix(T34):
    M = np.empty((4, 4))
    M[:3, :3] = T34[:3, :3]
    M[:3, 3] = T34[:3, 3]
    M[3, :] = [0, 0, 0, 1]
    return M

def write_ply(fn, verts, colors):
    ply_header = '''ply
    format ascii 1.0
    element vertex %(vert_num)d
    property float x
    property float y
    property float z
    property uchar red
    property uchar green
    property uchar blue
    end_header
    '''
    verts = verts.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    verts = np.hstack([verts, colors])
    with open(fn, 'w') as f:
        f.write(ply_header % dict(vert_num=len(verts)))
        np.savetxt(f, verts, '%f %f %f %d %d %d')


K = np.array([[7.070912e+02, 0.000000e+00, 6.018873e+02], [0.000000e+00, 7.070912e+02, 1.831104e+02], [0.000000e+00, 0.000000e+00, 1.000000e+00]])
baseline = 0.53790448812
win_size = 3
f = K[0,0]
T = np.eye(4)
ground_truth =get_ground_truth()
for i in range(21):
    l_image = get_LeftImages(i)
    r_image = get_RightImages(i)
    shape = l_image.shape 

    stereo = cv2.StereoSGBM_create(minDisparity= 16,numDisparities = 80,blockSize = 5,uniquenessRatio = 10,speckleWindowSize = 1,speckleRange = 32,disp12MaxDiff = 1,P1 = 8*3*win_size**2,P2 =32*3*win_size**2) 

    disparity = stereo.compute(l_image,r_image)
    q = np.array([[baseline, 0, 0, 0],[0, baseline, 0, 0],[0, 0, baseline*f, 0],[0, 0, 0, -1]])

    cv2.imwrite("disparity_map"+str(i)+".png",disparity-100)
    camera_pts = []
    temp1 = []
    q = np.array([[baseline, 0, 0, 0],[0, baseline, 0, 0],[0, 0, baseline*f, 0],[0, 0, 0, -1]])
    for j in range(disparity.shape[0]):
        for k in range(disparity.shape[1]):
            temp = np.array([k-(shape[1]/2),j-(shape[0]/2),1,disparity[j,k]])
            temp1.append([k-(shape[1]/2),j-(shape[0]/2)])

            temp = np.reshape(temp,(4,1))
            temp = np.matmul(q,temp)
            camera_pts.append(temp)


    camera_pts = np.asarray(camera_pts)
    temp1 = np.asarray(temp1)
    camera_pts = np.reshape(camera_pts,(camera_pts.shape[0],camera_pts.shape[1]))

    #T= get_transformation_matrix(ground_truth[i].reshape(3,4))
    camera_pts = np.dot(T,camera_pts.T)

    
    camera_pts = camera_pts/camera_pts[3,:]
    camera_pts = camera_pts.T

    camera_pts = camera_pts[:,:3]
    print("Camera pts shape",camera_pts.shape)
    

    mask_map = disparity > disparity.min()
    colors = cv2.cvtColor(l_image, cv2.COLOR_BGR2RGB)
    output_colors = colors[mask_map]
    mask_map = mask_map.ravel()
    output_points = camera_pts[mask_map]
    temp1 = temp1[mask_map]
    print("masked image array shape",temp1.shape)


    if i == 0 :
        total_stack = output_points
        total_col = output_colors
    else :
        total_stack = np.vstack((total_stack,output_points))
        total_col = np.vstack((total_col,output_colors))
        
    print('output pts and colors shape',output_points.shape,output_colors.shape)
    output_file = 'out_camera'+str(i)+'.ply'
    write_ply(output_file,output_points, output_colors)
    
    np.savetxt('temp'+str(i)+'.txt', temp1, delimiter=' ')

Camera pts shape (453620, 3)
masked image array shape (356828, 2)
output pts and colors shape (356828, 3) (356828, 3)
Camera pts shape (453620, 3)
masked image array shape (358718, 2)
output pts and colors shape (358718, 3) (358718, 3)
Camera pts shape (453620, 3)
masked image array shape (349554, 2)
output pts and colors shape (349554, 3) (349554, 3)
Camera pts shape (453620, 3)
masked image array shape (350427, 2)
output pts and colors shape (350427, 3) (350427, 3)
Camera pts shape (453620, 3)
masked image array shape (350343, 2)
output pts and colors shape (350343, 3) (350343, 3)
Camera pts shape (453620, 3)
masked image array shape (345008, 2)
output pts and colors shape (345008, 3) (345008, 3)
Camera pts shape (453620, 3)
masked image array shape (352616, 2)
output pts and colors shape (352616, 3) (352616, 3)
Camera pts shape (453620, 3)
masked image array shape (356406, 2)
output pts and colors shape (356406, 3) (356406, 3)
Camera pts shape (453620, 3)
masked image array shape (3

In [2]:
print(temp.shape)
print(temp1.shape)

(4, 1)
(326615, 2)


# Plotting camera coordinates
---

In [3]:
import numpy as np
import open3d as o3d  

def main():
    print("Load a ply point cloud, print it, and render it")
    pcd = o3d.io.read_point_cloud("out_camera0.ply")
    print(np.asarray(pcd.points))
    o3d.visualization.draw_geometries([pcd]) 

if __name__ == "__main__":
    main()

Load a ply point cloud, print it, and render it
[[ 0.39426   0.141353 -0.540266]
 [ 0.326675  0.117349 -0.448523]
 [ 0.32451   0.116799 -0.446417]
 ...
 [-0.552492 -0.167753 -0.644657]
 [-0.553403 -0.167753 -0.644657]
 [-0.554315 -0.167753 -0.644657]]


# Ploting the 3d scene (all combined) for camera
---

In [4]:
print(total_stack.shape)
print(total_col.shape)
output_points= total_stack
output_colors = total_col
output_file = 'out_c.ply'
write_ply(output_file,output_points, output_colors)
print(output_points.shape,output_colors.shape)

(7146680, 3)
(7146680, 3)
(7146680, 3) (7146680, 3)


In [5]:
def main():
    print("Load a ply point cloud, print it, and render it")
    pcd = o3d.io.read_point_cloud("out_c.ply")
    print(np.asarray(pcd.points))
    o3d.visualization.draw_geometries([pcd]) 

if __name__ == "__main__":
    main()

Load a ply point cloud, print it, and render it
[[ 0.39426   0.141353 -0.540266]
 [ 0.326675  0.117349 -0.448523]
 [ 0.32451   0.116799 -0.446417]
 ...
 [-0.273    -0.082755 -0.318016]
 [-0.272311 -0.08241  -0.316692]
 [-0.272759 -0.08241  -0.316692]]


# World point coordinates
---

In [6]:
import numpy as np
import cv2
import os

def get_LeftImages(i):
    return cv2.imread('./images/img2/'+str(i)+'.png')

def get_RightImages(i):
    return cv2.imread('./images/img3/'+str(i)+'.png')

def get_ground_truth():
    file = 'poses.txt'
    return np.genfromtxt(file, delimiter=' ',dtype=None)

def get_transformation_matrix(T34):
    M = np.empty((4, 4))
    M[:3, :3] = T34[:3, :3]
    M[:3, 3] = T34[:3, 3]
    M[3, :] = [0, 0, 0, 1]
    return M

def write_ply(fn, verts, colors):
    ply_header = '''ply
    format ascii 1.0
    element vertex %(vert_num)d
    property float x
    property float y
    property float z
    property uchar red
    property uchar green
    property uchar blue
    end_header
    '''
    verts = verts.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    verts = np.hstack([verts, colors])
    with open(fn, 'w') as f:
        f.write(ply_header % dict(vert_num=len(verts)))
        np.savetxt(f, verts, '%f %f %f %d %d %d')


K = np.array([[7.070912e+02, 0.000000e+00, 6.018873e+02], [0.000000e+00, 7.070912e+02, 1.831104e+02], [0.000000e+00, 0.000000e+00, 1.000000e+00]])
baseline = 0.53790448812
win_size = 3
f = K[0,0]

T = np.eye(4)
ground_truth =get_ground_truth()
for i in range(21):
    l_image = get_LeftImages(i)
    r_image = get_RightImages(i)
    shape = l_image.shape 

    stereo = cv2.StereoSGBM_create(minDisparity= 16,numDisparities = 80,blockSize = 5,uniquenessRatio = 10,speckleWindowSize = 1,speckleRange = 32,disp12MaxDiff = 1,P1 = 8*3*win_size**2,P2 =32*3*win_size**2) 
    disparity = stereo.compute(l_image,r_image)/16.0
    disparity = disparity/80.0
    q = np.array([[baseline, 0, 0, 0],[0, baseline, 0, 0],[0, 0, baseline*f, 0],[0, 0, 0, -1]])

    cv2.imwrite("disparity_map"+str(i)+".png",disparity-100)
    camera_pts = []
    temp1 = []
    for j in range(disparity.shape[0]):
        for k in range(disparity.shape[1]):

            temp = np.array([k,j,1,disparity[j,k]])
            temp1.append(np.array([k,j]))
            temp = np.reshape(temp,(4,1))
            temp = np.matmul(q,temp)
            camera_pts.append(temp)


    camera_pts = np.asarray(camera_pts)
    temp1 = np.asarray(temp1)
    camera_pts = np.reshape(camera_pts,(camera_pts.shape[0],camera_pts.shape[1]))
#     print("World points shape",camera_pts.shape)


    T0= get_transformation_matrix(ground_truth[i].reshape(3,4))
    T= get_transformation_matrix(ground_truth[i].reshape(3,4))
    camera_pts = np.dot(T,camera_pts.T)

    
    camera_pts = camera_pts/camera_pts[3,:]
    camera_pts = camera_pts.T
    camera_pts = camera_pts[:,:3]

    

#     print("World points shape final",camera_pts.shape)
    

    mask_map = disparity > disparity.min()
    colors = cv2.cvtColor(l_image, cv2.COLOR_BGR2RGB)
    output_colors = colors[mask_map]
    
    mask_map = mask_map.ravel()

    output_points = camera_pts[mask_map]
    temp1 = temp1[mask_map]
#     print("masked image array shape",temp1.shape)


    if i == 0 :
        total_stack = output_points
        total_col = output_colors
    else :
        total_stack = np.vstack((total_stack,output_points))
        total_col = np.vstack((total_col,output_colors))
#     print('output points, colors shape',output_points.shape,output_colors.shape)
    output_file = 'out_world'+str(i)+'.ply'
    write_ply(output_file,output_points, output_colors)

    
    np.savetxt('temp'+str(i)+'.txt', temp1, delimiter=' ')

# Plotting output world
---

In [7]:
import numpy as np
import open3d as o3d  

def main():
    print("Load a ply point cloud, print it, and render it")
    pcd = o3d.io.read_point_cloud("out_world0.ply")
    print(np.asarray(pcd.points))
    o3d.visualization.draw_geometries([pcd])

if __name__ == "__main__":
    main()

Load a ply point cloud, print it, and render it
[[ 183.488652  -30.440656  644.689787]
 [ 121.266083  -24.987338  544.077758]
 [ 120.552764  -24.894523  541.44407 ]
 ...
 [1422.999763 -520.486973  209.388427]
 [1424.061544 -520.535027  208.906606]
 [1425.123326 -520.583082  208.424785]]


# Ploting the 3d scene (all combined) for world
---

In [8]:
print(total_stack.shape)
print(total_col.shape)
output_points= total_stack
output_colors = total_col
output_file = 'out_w.ply'
write_ply(output_file,output_points, output_colors)
print(output_points.shape,output_colors.shape)

(7146680, 3)
(7146680, 3)
(7146680, 3) (7146680, 3)


In [9]:
def main():
    print("Load a ply point cloud, print it, and render it")
    pcd = o3d.io.read_point_cloud("out_w.ply")
    print(np.asarray(pcd.points))
    o3d.visualization.draw_geometries([pcd]) 

if __name__ == "__main__":
    main()

Load a ply point cloud, print it, and render it
[[ 183.488652  -30.440656  644.689787]
 [ 121.266083  -24.987338  544.077758]
 [ 120.552764  -24.894523  541.44407 ]
 ...
 [ 207.343969 -254.234186  740.898573]
 [ 206.200994 -253.193095  738.29292 ]
 [ 206.694521 -253.218361  738.583525]]
